# **Sentiment Analysis on COVID-19 related Tweets (Deep Learning: LSTM with GloVe: Global Vectors for Word Representation)**

*   Dataset: SentimentAnalysisCOVID-19MasterFinalDataset.csv
*   Runtime Type: GPU

# Mounting

In [ ]:
# We have to mount
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


# Import Libraries

In [ ]:
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import nltk
import re
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Reading the Dataset with Pandas

In [ ]:
#Read CSV from Pandas for Data Analysis
import pandas as pd

f = open('gdrive/My Drive/Colab Notebooks/SentimentAnalysisCOVID-19MasterFinalDataset.csv','rU')   
df = pd.read_csv(f)
df = df[['Translated','Sentiment']]
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 'U' mode is deprecated
  after removing the cwd from sys.path.


,Translated,Sentiment
0,me to covid,Neutral
1,so many realizations during ecq because of cov...,Positive
2,it's like a covid covid kesa back to cejay hah...,Positive
3,while we are all fighting against covid meanwh...,Neutral
4,person house because of covid,Neutral


Shape of the Dataset

In [ ]:
print(df.shape)

(44709, 2)


Convert Word Label to Numerical Label





In [ ]:
def convert_label(polarity):
  polarity.lower()
  for i in range(len(df)):
    if polarity == "Negative":
      return 0;
    elif polarity == "Neutral":
      return 1;
    elif polarity == "Positive":
      return 2;

df["Analysis"] = df["Sentiment"].apply(convert_label) 
df.reset_index(drop=True, inplace=True)

In [ ]:
df

,Translated,Sentiment,Analysis
0,me to covid,Neutral,1
1,so many realizations during ecq because of cov...,Positive,2
2,it's like a covid covid kesa back to cejay hah...,Positive,2
3,while we are all fighting against covid meanwh...,Neutral,1
4,person house because of covid,Neutral,1
...,...,...,...
44704,from as of june pm confirmed cases of covid...,Positive,2
44705,Boi Dabest really listen to martal covid covid...,Positive,2
44706,The worst Teacher is the one who is boggy with...,Negative,0
44707,Chaka Na Nko The Effect of Covid Does I say I ...,Neutral,1


# Cleaning of the Data

Stop Words

In [ ]:
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Cleaning the noise of the data

In [ ]:
def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9]+','',text) #remove @mentions
  text = re.sub(r'#','', text) #removing the '#' symbol
  text = re.sub(r'RT[\s]+','', text) #Removing RT
  text = re.sub(r'https?:\/\/\S+', '',text) #remove the hyper link
  text = re.sub('([^A-Za-z\ ])','',text) #removes everything and keeps only letters
  text = text.lower() #transforms everything to lowercase
  text = [word for word in text.split() if text.lower() not in stopword] #removes stopwords
  text = ' '.join(text)
  return text

df['Translated'] = df['Translated'].apply(cleanTxt)
df.head()

,Translated,Sentiment,Analysis
0,me to covid,Neutral,1
1,so many realizations during ecq because of cov...,Positive,2
2,its like a covid covid kesa back to cejay haha...,Positive,2
3,while we are all fighting against covid meanwh...,Neutral,1
4,person house because of covid,Neutral,1


Generate the Length of each row in the Dataset

In [ ]:
df['length'] = df['Translated'].apply(lambda x: len(x))
df = df[['Translated', 'Analysis','length']]

df.head()

,Translated,Analysis,length
0,me to covid,1,11
1,so many realizations during ecq because of cov...,2,153
2,its like a covid covid kesa back to cejay haha...,2,50
3,while we are all fighting against covid meanwh...,1,106
4,person house because of covid,1,29


Count number of occurences of each word

In [ ]:
#count number of occurences of each word
counts = Counter()
for index, row in df.iterrows():
    counts.update(row['Translated'])

In [ ]:
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

Encode the Data from Words to Numbers

In [ ]:
def encode_sentence(text, vocab2index, N=42):
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in text])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [ ]:
df['Encoded'] = df['Translated'].apply(lambda x: np.array(encode_sentence(x,vocab2index)))
df.head(15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


,Translated,Analysis,length,Encoded
0,me to covid,1,11,"[[2, 3, 4, 5, 6, 4, 7, 6, 8, 9, 10, 0, 0, 0, 0..."
1,so many realizations during ecq because of cov...,2,153,"[[11, 6, 4, 2, 12, 13, 14, 4, 15, 3, 12, 16, 9..."
2,its like a covid covid kesa back to cejay haha...,2,50,"[[9, 5, 11, 4, 16, 9, 23, 3, 4, 12, 4, 7, 6, 8..."
3,while we are all fighting against covid meanwh...,1,106,"[[24, 25, 9, 16, 3, 4, 24, 3, 4, 12, 15, 3, 4,..."
4,person house because of covid,1,29,"[[26, 3, 15, 11, 6, 13, 4, 25, 6, 18, 11, 3, 4..."
5,i will die in stress not in covid,1,33,"[[9, 4, 24, 9, 16, 16, 4, 10, 9, 3, 4, 9, 13, ..."
6,i want to join case there is still a covid,1,42,"[[9, 4, 24, 12, 13, 5, 4, 5, 6, 4, 27, 6, 9, 1..."
7,so confiscate the akuang alcohol ml then the w...,1,62,"[[11, 6, 4, 7, 6, 13, 22, 9, 11, 7, 12, 5, 3, ..."
8,i choose you to be a positive from covid charrr,2,47,"[[9, 4, 7, 25, 6, 6, 11, 3, 4, 14, 6, 18, 4, 5..."
9,jgh then ligo grabe covid stop na,1,33,"[[27, 19, 25, 4, 5, 25, 3, 13, 4, 16, 9, 19, 6..."


Check the balance of the whole labelling

In [ ]:
#check how balanced the dataset is
Counter(df['Analysis'])

Counter({0: 7707, 1: 17520, 2: 19482})

# **Model Preparation**

In [ ]:
X = list(df['Encoded'])
y = list(df['Analysis'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# **Python to Pytorch**

In [ ]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx): 
        return torch.from_numpy(self.X[idx][0].astype(np.int32)).to(device), self.y[idx], self.X[idx][1]

In [ ]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [ ]:
train_loss=list()
validation_loss=list()
accuracy_val=list()
prediction=list()

# **Create function for Train Model and Validation Metrics**

In [ ]:
def train_model(pretrained_weights,model, epochs, lr,saving_path):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    #Utilization of Stochastic Gradient Descent for optimization
    #with corresponding Learning Rate
    best_valid_loss = float('inf')
    optimizer = torch.optim.SGD(parameters, lr=lr)
    best_epoch=0
    for i in range(epochs):
        sum_loss = 0.0
        total = 0
        correct=0.0
        for x, y, l in train_dl:
            # x is the data
            # y is the target variable (true label)
            # l is the label

            x = x.long()
            y = y.long().to(device)

            #prediction of the model using the given data
            y_pred = model(x, l)
            pred = torch.max(y_pred, 1)[1]

            #resetting gradients for each iteration
            optimizer.zero_grad()
            #using Cross Entropy for Loss calculation
            loss = F.cross_entropy(y_pred, y).to(device)
            #backward propagation
            loss.backward()
            #gradient step
            optimizer.step()
            #calculation of total loss value
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
            correct += (pred == y).float().sum()
        #return values from the validation metrics    
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)

        #gathering the data for train loss, validation loss, accuracy
        #for each epoch
        train_loss.append(sum_loss/total)
        validation_loss.append(val_loss)
        accuracy_val.append(val_acc)

        print("Epoch "+str(i+1)+": train loss %.3f, val loss %.3f, train accuracy %.3f, val accuracy %.3f, and val rmse %.3f" % 
              (sum_loss/total, val_loss, correct/total, val_acc, val_rmse))
        
        if val_loss < best_valid_loss:
          best_valid_loss = val_loss
          best_val_acc=val_acc
          best_epoch=i
          torch.save({'epoch': i,
            'pretrained_weights':pretrained_weights,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'train_loss': sum_loss/total,
            'train_acc': correct/total,
            'val_acc':val_acc}
            , saving_path)
          
        if abs((sum_loss/total)-val_loss) >= 0.2:
          return best_epoch
    return best_epoch

def validation_metrics (model, valid_dl):
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
      # x is the data
      # y is the target variable (true label)
      # l is the label
      x = x.long()
      y = y.long().to(device)

      #prediction of the model using the given data
      y_hat = model(x, l)

      #using Cross Entropy for Loss calculation
      loss = F.cross_entropy(y_hat, y).to(device)
      pred = torch.max(y_hat, 1)[1]

      #collecting the prediction data for y_pred use in classification report
      prediction.append(pred)

      #calculation of total loss value and RMSE
      correct += (pred == y).float().sum()
      total += y.shape[0]
      sum_loss += loss.item()*y.shape[0]
      sum_rmse += np.sqrt(mean_squared_error(pred.cpu(), y.cpu().unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [ ]:
batch_size = 128
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

# Pre-trained Vectors

In [ ]:
def load_glove_vectors(glove_file="/content/gdrive/MyDrive/Colab Notebooks/glove.6B.100d.txt"): #50, 100, 300
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

# Embedding Size

In [ ]:
def get_emb_matrix(pretrained, word_counts, emb_size = 100): #50, 100, 300
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx

In [ ]:
word_vecs = load_glove_vectors()
pretrained_weights, vocab, vocab2index = get_emb_matrix(word_vecs, counts)

# LSTM GloVe Word Embedding

In [ ]:
class LSTM_glove_vecs(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim,num_layers, hidden_dim, dropout, glove_weights,bidirectional) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        #copying GloVe word embedding to model
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights).to(device)) 
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        #LSTM module
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers, batch_first=True, bidirectional=bidirectional)
        #Linear module for the final prediction 
        #of which emotion class is the processed text
        self.linear = nn.Linear(hidden_dim, 3)
        #Dropout Regularization for LSTM
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

# Declaring parameters for LSTM GloVe Word Embedding

In [ ]:
final_path='/content/gdrive/MyDrive/Best_Model_LSTM_Thesis.pt'

embedding=100

hidden_layers=128

num_layers=3

epoch=2000 
dropout=0.3
bidirectional=True

#Loading the model with the GloVe Word Embedding and given parameters
model = LSTM_glove_vecs(vocab_size, 
                        embedding, 
                        num_layers, 
                        hidden_layers,
                        dropout, 
                        pretrained_weights,
                        bidirectional)

In [ ]:
model.to(device)

LSTM_glove_vecs(
  (embeddings): Embedding(29, 100, padding_idx=0)
  (lstm): LSTM(100, 128, num_layers=3, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=3, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

# Train Model

In [ ]:
stop_epoch=train_model(pretrained_weights, model, epochs=epoch, lr=0.6, saving_path=final_path)

Epoch 1: train loss 1.033, val loss 1.034, train accuracy 0.428, val accuracy 0.441, and val rmse 1.039
Epoch 2: train loss 1.032, val loss 1.033, train accuracy 0.430, val accuracy 0.441, and val rmse 1.039
Epoch 3: train loss 1.031, val loss 1.031, train accuracy 0.435, val accuracy 0.451, and val rmse 1.016
Epoch 4: train loss 1.029, val loss 1.029, train accuracy 0.441, val accuracy 0.456, and val rmse 0.988
Epoch 5: train loss 1.028, val loss 1.028, train accuracy 0.445, val accuracy 0.463, and val rmse 0.956
Epoch 6: train loss 1.025, val loss 1.023, train accuracy 0.454, val accuracy 0.473, and val rmse 0.991
Epoch 7: train loss 1.021, val loss 1.022, train accuracy 0.466, val accuracy 0.501, and val rmse 0.965
Epoch 8: train loss 1.030, val loss 1.031, train accuracy 0.433, val accuracy 0.451, and val rmse 1.028
Epoch 9: train loss 1.029, val loss 1.030, train accuracy 0.440, val accuracy 0.453, and val rmse 1.009
Epoch 10: train loss 1.028, val loss 1.029, train accuracy 0.445

KeyboardInterrupt: ignored

In [ ]:
#Getting the y values (label)
targets=list()
for x,y,l in val_dl:
  for z in y:
    targets.append(z)

In [ ]:
temp = 0
for i in prediction:
  temp+=1
  if(len(i)< 128):
    temp+=1
    break
print('prediction model indexing for validation', temp-1)

In [ ]:
preds=list()
temp=prediction[(stop_epoch*88):(stop_epoch*88)+88]
for i in range (0,len(temp)):
  for z in temp[i]:
    preds.append(z.cpu().numpy())

# Evaluation Metrics

In [ ]:
#Confusion Matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_true = targets
y_pred = preds
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))